<a href="https://colab.research.google.com/github/sahilnale/2024-sp-stats21/blob/main/sentiment_analysis_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment-140 Classification

In this notebook, we will explore the application of transformers for sentiment analysis of different tweets. Our goal is to classify tweets to have either positive or negative sentiment. We will first consider the performance of a pre-trained BERT model, finetuned on the Sentiment-140 dataset, and later we'll develop and train our own transformer architecture from scratch. Let's dive right in!

First of all, we need a couple of useful libraries that you're hopefully familiar with.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# this automatically reloads the libraries so you can update them dynamically
%load_ext autoreload
%autoreload 2

## Loading our data

Let's start by preparing our dataset. We have implemented the dataset class for you in `data_utils/SentimentDataset.py`, feel free to check the implementation. However, before we can use the `SentimentDataset` class to create our train data and test data objects, we need to pre-process the raw data.

You can download raw data from [here](https://www.kaggle.com/datasets/kazanova/sentiment140). If you examine the raw data file, you can see that there is a lot of redundant information such as the time of each tweet or usernames. For our sentiment analysis, we simply need the tweets and ground truth labels. To preprocess the data file, simply paste the original CSV file in the root project directory without changing the file name and run the script `preprocess_dataset.py`. This should create a new CSV file called `dataset.csv`. (The reason why we do not include the data files on the GitHub repository is that they are simply too big.) Then, you're ready to create a dataset and dataloader below.

Notice that the original dataset contains 1.6 million tweets; that is huge! Therefore, it may take forever to train on so much data, especially if you happen not to have access to a fast GPU (talk to the officers about getting access to the ACM AI server!). Therefore, in the next cell you can configure whether to use the whole dataset or just a subset of it.

In [ ]:
import torch
import csv
import numpy as np
from torch.utils.data import Dataset

class SentimentDataset(Dataset):

    def __init__(self, csv_path, training_set=True):
        tweets = []
        labels = []
        with open(csv_path, 'r') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                tweets.append(row[0])
                labels.append(int(row[1]))

        total_tweets = len(tweets)
        train_indices = np.random.choice(total_tweets, int(0.8 * total_tweets), replace=False)
        test_indices = np.setdiff1d(np.arange(total_tweets), train_indices)

        if training_set:
            self.tweets = [tweets[i] for i in train_indices]
            self.labels = [labels[i] for i in train_indices]
        else:
            self.tweets = [tweets[i] for i in test_indices]
            self.labels = [labels[i] for i in test_indices]

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        return self.tweets[idx], self.labels[idx]

In [ ]:
use_subset_of_data = True
subset_percentage = 0.05 # set the percentage of original dataset size you'd like to use

In [ ]:
train_data = SentimentDataset('dataset.csv', training_set=True)
test_data = SentimentDataset('dataset.csv', training_set=False)

if use_subset_of_data:
    train_set_len = int(subset_percentage * len(train_data))
    test_set_len = int(subset_percentage * len(test_data))

    train_subset_indices = np.random.choice(len(train_data), size=train_set_len, replace=False)
    train_sampler = SubsetRandomSampler(train_subset_indices)

    test_subset_indices = np.random.choice(len(test_data), size=test_set_len, replace=False)
    test_sampler = SubsetRandomSampler(test_subset_indices)

    train_loader = DataLoader(train_data, batch_size=32, sampler=train_sampler)
    test_loader = DataLoader(test_data, batch_size=32, sampler=test_sampler)

else:
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

Let's examine a sample tweet from the dataset and print its corresponding ground truth label. Label 1 corresponds to positive sentiment, while label 0 stands for negative sentiment. You can run the cell below multiple times to see different tweets.

In [ ]:
tweets, labels = next(iter(train_loader))
print("Sample tweet: ", tweets[0])
print("Ground truth label: ", labels[0].numpy())

## Fine-tuning BERT

First, let's examine the performance of a pre-trained BERT model on the sentiment analysis task. BERT (original paper [here](https://arxiv.org/abs/1810.04805)) is a language representation model released by Google in 2018 that uses a Bidirectional Transformer architecture.

It can be easily fine-tuned on downstream tasks such as sentiment analysis by appending a classification layer to the original BERT architecture. This is already done automatically by importing the `BertForSequenceClassification` from the Hugging Face library called `transformers`. The newly added classification layer is untrained (as you'll see in the warning that appears when you run the following cell) and so we have to fine-tune on the Sentiment-140 data. We will also import a pre-trained tokenizer, which is necessary to convert inputs from natural language to a numerical representation machine learning models can understand. The tokenizer imported from Hugging Face uses WordPiece tokenization, you can learn more about the algorithm [here](https://huggingface.co/learn/nlp-course/en/chapter6/6).

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
pretrained_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Below, we define a `preprocess` function that takes in the tweets and tokenizes them. The tokenizer will break down words into smaller units called subword tokens and map these words to numbers (IDs), which can be processed by the model. Furthermore, it will add special tokens to the sequence that will normalize inputs to the model.

The `preprocess` function return `input_ids` which represents the tokenized input sequence and `attention_mask` which specifies to the model which tokens to attend to while computing self-attention (more on that later!).

In [ ]:
def preprocess(tweets):

    encoded_batch = tokenizer.batch_encode_plus(
        tweets,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded_batch['input_ids']
    attention_masks = encoded_batch['attention_mask']

    return input_ids, attention_masks

There are three different types of special tokens normally used by the BERT tokenizer. `[CLS]` token is added at the beginning of the input sequence. The final hidden state corresponding to the `[CLS]` token is typically used as the representation of the entire input sequence for classification tasks, so it will be very useful to us later for sentiment analysis. The `[SEP]` token is used to separate different segments of the input sequence. It helps the model distinguish between different parts of the input and process them accordingly. In our case, one `[SEP]` token will typically be added at the end of the tokenized input sequence (tweet). Finally, the `[PAD]` token is used for padding shorter input sequences to a fixed length, so that they can be easily processed in batches without any shape mismatch issues. With BERT, we would typically pad the input sequence to reach a total length of 512 tokens; however, since we're analyzing tweets that are by definition very short, using maximum sequence length of 128 tokens should suffice.

Below, you can see what the IDs reserved for special tokens are.

In [ ]:
cls_token_id = tokenizer.cls_token_id
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id

print("[CLS] Token ID: " + str(cls_token_id) + ", [SEP] Token ID: " + str(sep_token_id) + ", [PAD] Token ID: " + str(pad_token_id))

We can print a sample tweet and its corresponding tokenized representation. Notice that each input sequence starts with token ID 101 (CLS token), ends with token ID 102 (SEP token), and is padded with token ID 0 (PAD token).

In [ ]:
tweets, labels = next(iter(train_loader))
print("Sample tweet: ", tweets[0], '\n')
input_ids, att_masks = preprocess(tweets)
print("Tokenized representation: ", input_ids[0])

Next, we need to define a training and evaluation loop. Notice that we use the `preprocess` function in both the `train` and `evaluate` to transform the natural language input to tokenized representation. We will also collect loss and accuracy statistics during training, so that we can plot the training history later.

Note that the `finetuning` parameter is used simply because the internal Hugging Face representation of the pre-trained model is slightly different from what our representation will be when we later define our own transformer model. Thus, when using the training loop for fine-tuning the pre-trained downloaded from Hugging Face, set `finetuning=True`, otherwise set it to `False`.

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, device,
          num_epochs, finetuning=False):

    # Place model on device
    model = model.to(device)

    loss_history = []
    acc_history = []

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader),
                  desc=f'Epoch {epoch + 1}/{num_epochs}',
                  position=0,
                  leave=True) as pbar:

            for inputs, labels in train_loader:

                input_ids, attention_masks = preprocess(inputs)
                # Move inputs and labels to device
                input_ids = input_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.to(device)

                # Zero out gradients
                optimizer.zero_grad()

                # Compute the logits and loss
                outputs = model(input_ids=input_ids, attention_mask=attention_masks)
                if finetuning:
                    outputs = outputs.logits
                loss = criterion(outputs, labels)

                # Backpropagate the loss
                loss.backward()

                # Update the weights
                optimizer.step()

                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())

                loss_history.append(loss.item())


        avg_loss, accuracy = evaluate(model, val_loader, criterion, device, finetuning)
        print(
            f'Test set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}'
        )
        acc_history.append(accuracy)

    return loss_history, acc_history


def evaluate(model, test_loader, criterion, device, finetuning=False):

    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0

        for inputs, labels in test_loader:

            input_ids, attention_masks = preprocess(inputs)
            # Move inputs and labels to device
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            labels = labels.to(device)

            # Compute the logits and loss
            outputs = model(input_ids=input_ids, attention_mask=attention_masks)
            if finetuning:
                outputs = outputs.logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(outputs, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(labels)

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples

    return avg_loss, accuracy

Make sure you are running the model on CUDA.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

We will finetune all the weights in the BERT model (including the pre-trained weights and the newly initialized classifier weights). An alternative approach would be to freeze the pre-trained weights and only fine-tune the classifier weights. You can do that by passing in `model.classifier.parameters()` to the optimizer instead of passing in all the weights of the model.

In [ ]:
optimizer = torch.optim.AdamW(pretrained_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

loss_history, acc_history = train(pretrained_model, train_loader, test_loader, optimizer, criterion, device, num_epochs=1, finetuning=True)

# Let's save our trained weights
torch.save(pretrained_model.state_dict(), 'bert_fine_tuned.pt')

Next, let's plot our training loss and test accuracy history that documents the fine-tuning process.

In [ ]:
plt.plot(loss_history)
plt.xlabel('Iterations')
plt.ylabel('Cross-entropy Loss')
plt.title('BERT Fine-tuning, Loss history')
plt.show()

## Creating our own Transformer architecture from scratch

That was fun, right? But maybe slightly too easy, don't you think? Let's try to implement our own transformer architecture now, without the help of pre-trained models. We will base our architecture off of BERT, which will also allow us to effectively compare the performance of our own implementation with the pre-trained model. The high-level of the model architecture can be seen below.

Note that, compared to illustration below, all the tensor sizes will most likely be different and specific to the hyperparameters of our model.

![BERT For Sequence Classification](https://github.com/uclaacmai/Tweets-Are-All-You-Need/blob/main/bert_classification.png?raw=true)

![BERT Encoder](https://github.com/uclaacmai/Tweets-Are-All-You-Need/blob/main/bert_encoder.png?raw=true)

Let's first define some hyperparameters of our network. (You can also play around with these later to improve your model's performance.)

In [ ]:
MAX_SEQ_LEN = 128 # Maximum length of tokenized input sequence (we already covered this above!)
VOCAB_SIZE = 30522 # number of different token -> ID mappings, specific to the BERT Tokenizer (do not change)
N_LAYERS = 12 # number of transformer layers stacked on top of each other
N_HEADS = 12 # number of heads of the transformer, input sequence will be split and equally distributed to each head
EMB_SIZE = 768 # refers to the dimensionality of the vector representations used to encode input tokens
INTERMEDIATE_SIZE = EMB_SIZE * 4 # dimensionality of output of the Intermediate layer
DROPOUT = 0.1 # probability of dropping a neuron during training
N_CLASSES = 2 # we are classifying the sentiment of each tweet simply as POSITIVE or NEGATIVE
LAYER_NORM_EPS = 1e-12 # value used in Layer Norm for numerical stability

## Transformers

Transformers are a type of neural network architecture primarily used in the field of natural language processing (NLP). Unlike previous models that processed inputs sequentially (like RNNs), transformers process whole sequences of data in parallel, which significantly speeds up training.

### Why are they important?

They have been highly successful in a variety of NLP tasks like translation, text summarization, and sentiment analysis due to their ability to handle long-range dependencies in text.

### Components of Transformers

1. Input Embeddings + Positional Encoding
2. Multi-Head Attention
3. Positional-Wise Feed-Forward Networks
4. Layer Normalization and Residual Connections


## 1. Input Embeddings

Converts tokens (words) into vectors of continuous numbers that the model can understand.  

*(Optional)* Check out 3Blue1Brown's video on Word Embeddings [here](https://www.youtube.com/watch?v=wjZofJX0v4M).

### Concept of Embeddings

Embeddings are vector representations of text, where words with **similar meanings** have **similar representations**. These vectors are learned and adjusted during the training of the model to optimize performance on a specific task.

### Importance of Embeddings

Embeddings capture the semantic properties of words, which means they convert the words/tokens into a form that a neural network can work with. This allows the model to **understand and process language** by **quantifying the similarities between different words or phrases**.

### Embeddings in Transformers

1. **Tokenization**:
   - **Process**: Text input is split into tokens (words or subwords), which are then converted into numerical IDs that correspond to entries in the embedding table.
   - **Role in Model**: This step is critical because it translates human-readable text into a format that can be mathematically manipulated by the model.

2. **Lookup Table**:
   - **Process**: Each token ID is used to retrieve its corresponding embedding vector from an embedding matrix (or table) that the model learns during training.
   - **Role in Model**: This embedding matrix acts as the foundational input layer of the transformer, providing a dense and informative representation for each token.

3. **Dimensionality**:
   - **Process**: Embeddings are usually vectors of fixed size (e.g., 256, 512 dimensions), regardless of the vocabulary size.
   - **Role in Model**: Higher dimensions generally capture more detailed semantic information about each token, but also increase model complexity and computational cost.

### Positional Embeddings

If we only turn words into embeddings, the model won't know the order of them in a sentence. We need to somehow preserve the positional info.

- **Concept**: Since transformers process words in parallel rather than sequentially, positional encodings are added to embeddings to give the model information about the order of words in a sentence.
- **Why Necessary**: Helps the model understand how the position of a word in a sentence affects its meaning.

### Training and Adjustments

**Learning Process**:
- Embeddings are not static; they are adjusted during the training process to minimize the model's prediction error. This learning happens through backpropagation, which adjusts the embeddings to better capture the relationships and contexts in which words appear.

**Impact on Performance**:
- The quality of embeddings significantly affects the model's performance. Better embeddings lead to a better understanding of language nuances, such as synonyms, antonyms, and different contexts.

### Toy Example

In [ ]:
import torch
import torch.nn as nn

# Vocabulary size and embedding dimension
vocab_size = 1000  # Number of unique tokens
embedding_dim = 64  # Size of each embedding vector

# Create an embedding layer
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Example token IDs
token_ids = torch.LongTensor([10, 200, 500])

# Get embeddings for token IDs
embeddings = embedding_layer(token_ids)
print(embeddings)

This example initializes an embedding layer with random values which then can be trained to learn meaningful representations through a task like sentiment analysis or machine translation.

### Your Turn

Now, let's implement the `EmbeddingLayer` class. This class will be responsible for converting tokens into dense vectors that the model can understand. We will use PyTorch's `nn.Embedding` module to create the embedding layer.

1. token embedding: vocab -> embedding.
2. positional embedding: position -> embedding
3. add them together
4. layer normalization
5. dropout

In [ ]:
class EmbeddingLayer(nn.Module):

    def __init__(self, vocab_size, emb_size, max_seq_length, layer_norm_eps=1e-12, dropout=0.1):
        super().__init__()

        # TODO: Initialize word embeddings
        self.word_embeddings = None
        self.position_embeddings = None
        self.ln = None
        self.dropout = None

        # sets self.position_ids to a tensor containing values from 0 to max_seq_length, reshaped to (1, max_seq_length)
        # register_buffer makes it so that this does NOT get updated during training
        self.register_buffer("position_ids", torch.arange(max_seq_length).expand((1, -1)))

    def forward(self, input_ids):
        pass

        return emb


## 2. Multi-Head Attention

Multi-head attention consists of several attention mechanisms (heads) running in parallel. Each head computes an attention score known as "Scaled Dot-Product Attention," which determines how much each element in a sequence should pay attention to every other element. Combining multiple heads allows the model to capture various aspects of semantic relationships in different subspaces.  

*(Optional)* Check out 3Blue1Brown's video on Attention Mechanism [here](https://www.youtube.com//watch?v=eMlx5fFNoYc).

### Why Multi-Head Attention

By using multiple attention heads, transformers can:
- Capture a richer diversity of relationships within the data.
- Focus on different parts of the sentence simultaneously.
- Improve the ability of the model to focus on relevant parts of the input for making predictions.

### Components of Multi-Head Attention

1. **Scaled Dot-Product Attention**: The basic attention mechanism computes the dot products of the query with all keys, divides each by the square root of the dimensionality (to stabilize gradients), applies a softmax function to obtain weights on the values, and finally returns an output vector.

2. **Multiple Heads**: Each head performs attention independently, allowing the model to focus on different features across different positions in the input sequence.

3. **Concatenation and Final Linear Transformation**: Outputs from individual heads are concatenated and passed through a final linear layer to combine the different learned aspects.

### Your Turn

We compute attention using the formula below, introduced in [this](https://arxiv.org/abs/1706.03762) famous paper.

![Attention](https://github.com/uclaacmai/Tweets-Are-All-You-Need/blob/main/attention.png?raw=true)

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, emb_size, n_heads, dropout=0.1, layer_norm_eps=1e-12):
        '''
        Args:
            emb_size (int): The size of the input embeddings
            n_heads (int): The number of attention heads
            dropout (float): The dropout probability
            layer_norm_eps (float): The epsilon value for layer normalization

        '''
        super().__init__()
        assert emb_size % n_heads == 0, "emb_size must be divisible by n_heads"

        # TODO: initialize head size, query, key, value, dropout, final_linear, and layer norm
        # you want to use nn.Linear, nn.Dropout, nn.LayerNorm
        # query, key, value, final_linear should map from emb_size to emb_size
        self.head_size = 0
        self.query = None
        self.key = None
        self.value = None
        self.dropout = None

        self.final_linear = None
        self.ln = None


    def forward(self, emb, att_mask):
        '''
        Args:
            emb (torch.Tensor): The input embeddings with shape (B, T, C)
            att_mask (torch.Tensor): The attention mask with shape (B, T)

        Returns:
            torch.Tensor: The output embeddings with shape (B, T, C)

        1. You want to obtain the query (Q), key (K), and value (V) tensors from the input embeddings
        2. Compute weights:
            a. find dot product of Q and K
            b. divide by the square root of the head size
            c. apply the attention mask
            d. apply softmax
        3. Compute attention: multiply weights by V
        4. Feed the attention through the final linear layer
        5. Apply dropout, residual connection, and layer normalization

        '''
        B, T, C = emb.shape  # batch size, sequence length, embedding size

        # TODO: 1. get the query, key, and value tensors
        # TODO: 2. rearrange the tensors from B x T x C to B x n_heads x T x head_size

        q =
        k =
        v =

        # TODO: compute the weights before attention mask
        weights =

        # set the pad tokens to -inf so that they equal zero after softmax
        if att_mask != None:
            att_mask = (att_mask > 0).unsqueeze(1).repeat(1, att_mask.size(1), 1).unsqueeze(1)
            weights = weights.masked_fill(att_mask == 0, float('-inf'))

        # apply softmax and dropout
        weights =

        # compute attention
        attention =

        # rearrange the attention tensor from B x n_heads x T x head_size to B x T x C

        # apply final linear layer, dropout, residual connection, and layer normalization
        out =

        return out


### Side Note: Layer Normalization and Residual Connections

- **Concept**: Techniques used within transformers to stabilize the learning process. Residual connections help in propagating gradients through deep networks without vanishing.
- **Why Necessary**: Improves training efficiency and model performance by preventing the vanishing gradient problem.

## 3. Position-Wise Feed-Forward Networks

The position-wise feed-forward network is a standard fully connected neural network that applies the same two linear transformations to each position independently and identically. It consists of two layers:
- The first linear transformation maps the input dimension `emb_size` to a dimension `intermediate_size` (typically much larger, e.g., 2048).
- The second linear transformation maps it back from `intermediate_size` to `emb_size`.

### Why Position-wise Feed-Forward Networks

This component of the transformer allows the model to consider each position separately and integrate non-linearity into the model. Each position in the encoder or decoder can be thought of as having its own little neural network that processes the information.

Its role is two-fold:
1. It increases the model's capacity by introducing additional learnable parameters.
2. It facilitates the progressive enrichment and refinement of token representations as they pass through each encoder layer.

### Your Turn

In [ ]:
class PositionWiseFeedForward(nn.Module):

    def __init__(self, emb_size, intermediate_size, dropout=0.1, layer_norm_eps=1e-12):
        '''
        Args:
            emb_size (int): The size of the input embeddings
            intermediate_size (int): The size of the intermediate layer
            dropout (float): The dropout probability
            layer_norm_eps (float): The epsilon value for layer normalization

        We want to create two fully connected layers mapping from emb_size to intermediate_size and back
        '''

        # TODO: initialize the two fully connected layers, gelu, dropout, and layer norm
        super().__init__()
        self.fc1 = None
        self.fc2 = None

        # we want some non-linearity between the two layers, we are using GELU, feel free to experiment with other activations
        self.gelu = None
        self.dropout = None
        self.ln = None

    def forward(self, att_out):
        '''
        Args:
            att_out (torch.Tensor): The output embeddings from the multi-head attention layer with shape (B, T, C)

        Returns:
            torch.Tensor: The output embeddings with shape (B, T, C)

        TODO:
        1. Apply the first fully connected layer and non-linearity
        2. Apply the second fully connected layer and dropout
        3. Apply a residual connection and layer normalization
        '''


        return out

## Transformer Block: Putting It All Together

Now that we have implemented the individual components of the transformer, we can combine them to create a single transformer block. This block will contain the following components:
- Multi-Head Attention
- Position-wise Feed-Forward Networks
- Layer Normalization Residual Connections

In [ ]:
class TransformerBlock(nn.Module):

    def __init__(self, emb_size, n_heads, intermediate_size, dropout=0.1, layer_norm_eps=1e-12):
        '''
        Args:
            emb_size (int): The size of the input embeddings
            n_heads (int): The number of attention heads
            intermediate_size (int): The size of the intermediate layer
            dropout (float): The dropout probability
            layer_norm_eps (float): The epsilon value for layer normalization

        We want to create a transformer block that consists of a multi-head attention layer and a position-wise feed forward layer.
        Note: We took care of layer normalization and residual connections in the previous MultiHeadAttention and PositionWiseFeedForward classes
        '''

        # TODO: initialization
        super().__init__()
        self.attn = None
        self.ff = None

    def forward(self, emb, att_mask):
        '''
        Args:
            emb (torch.Tensor): The input embeddings with shape (B, T, C)
            att_mask (torch.Tensor): The attention mask with shape (B, T)

        Returns:
            torch.Tensor: The output embeddings with shape (B, T, C)

        Take emb and att_mask and pass them through the multi-head attention layer and position-wise feed forward layer
        '''

        return out

## Your Own BERT

Now, you have everything you need to create your own (simplified) BERT* model. This model will consist of the following components:

1. Embedding Layer
2. Encoder Block (multiple transformer blocks stacked together)
3. Pooler (to aggregate information from the encoder)
4. Classifier (to predict the sentiment of the input text)

***Note**: we are not exactly replicating BERT, but rather creating a simplified version of it for educational purposes.

## Encoder Block

The power of BERT's encoder block comes from stacking multiple encoder layers together. Each layer outputs a higher-level representation of the input tokens. With each successive layer, the model can capture more complex dependencies and refine its understanding of the input context.

The **output** of the **last encoder** layer serves as a **comprehensive representation of the input sequence**, embedding both the immediate meaning and broader context of each token.

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self, emb_size, n_layers, n_heads, intermediate_size, dropout=0.1, layer_norm_eps=1e-12):
        '''
        Args:
            emb_size (int): The size of the input embeddings
            n_layers (int): The number of transformer blocks
            n_heads (int): The number of attention heads
            intermediate_size (int): The size of the intermediate layer
            dropout (float): The dropout probability
            layer_norm_eps (float): The epsilon value for layer normalization

        We want to create n_layers of transformer blocks
        '''
        super().__init__()

        # TODO: initialize transformer layers, (hint: checkout nn.ModuleList)
        self.transformer_layers = None

    def forward(self, emb, att_mask):
        '''
        Args:
            emb (torch.Tensor): The input embeddings with shape (B, T, C)
            att_mask (torch.Tensor): The attention mask with shape (B, T)

        Returns:
            torch.Tensor: The output embeddings with shape (B, T, C)

        Pass the input embeddings through each transformer block
        '''
        # TODO:

        return emb

Hang on tight, we're almost there! We have successfully implemented the BERT Encoder architecture, but now we have to tailor the model to a clasification task like sentiment analysis.

The output from the encoder is a matrix where each row represents an enriched vector representation for the corresponding input token, capturing contextual and positional information through the self-attention and feed-forward layers. However, for sequence classification tasks, a single vector representation summarizing the meaning of the entire input sequence is required, rather than individual token representations. To achieve this, we will only process the representation of the `[CLS]` token (always inserted at the start of our input sequence), as it is expected to contain a contextualized, high-level representation of the entire sequence and therefore is a good candidate to be pooled for sequence classification.

## Pooler

The pooler component in BERT takes the output of the final encoder layer and transforms it to create a fixed-size output vector. Typically, it processes the output corresponding to the [CLS] token, which is the first token in the sequence. This token is specially designed to hold the aggregate representation of the input sequence after passing through the transformer blocks.

### Why Pooler

The main purpose of the pooler is to condense the complex and variable-length output of the encoder into a fixed-size embedding that can be used efficiently by downstream tasks, such as classification layers. For tasks like sentence classification, the model needs a singular representation of the entire input, which the pooler provides by transforming the [CLS] token's final hidden state.


### Your Turn

Pooler involves 3 steps:
1. Extract the hidden state corresponding to the [CLS] token.
2. Apply a linear transformation to this hidden state: embedding_size -> embedding_size.
3. Apply a non-linear activation function (like Tanh)

In [ ]:
class Pooler(nn.Module):
    def __init__(self, emb_size):
        '''
        Args:
            emb_size (int): The size of the input embeddings
        '''
        super().__init__()

        # TODO: Define the linear layer and activation function
        self.dense = None
        self.activation = None

    def forward(self, encoder_out):
        '''
        Args:
            encoder_out (torch.Tensor): The output embeddings from the encoder with shape (B, T, C)

        Returns:
            torch.Tensor: The pooled embeddings with shape (B, C)
        '''

        # TODO: 1. extract the first token's embeddings
        # TODO: 2. apply the linear layer and activation function

        return out

## Final Simplified BERT Classifier

Now that we have the pooler component, we can add a classification layer on top of it to predict the sentiment of the input text. This classification layer will be a simple linear layer that takes the pooled output from the pooler and maps it to `n_classes` output units.

### Your Turn

In [ ]:
class SimplifiedBertClassifier(nn.Module):
    def __init__(self, vocab_size, emb_size, n_layers, n_heads, intermediate_size, dropout=0.1, n_classes=2):
        '''
        Args:
            vocab_size (int): The size of the vocabulary
            emb_size (int): The size of the input embeddings
            n_layers (int): The number of transformer blocks
            n_heads (int): The number of attention heads
            intermediate_size (int): The size of the intermediate layer
            dropout (float): The dropout probability
            n_classes (int): The number of classes
        '''
        super().__init__()

        # TODO: Initialization
        self.embedding = None
        self.encoder = None
        self.pooler = None
        self.dropout = None
        self.classifier = None

    def forward(self, input_ids, attention_mask=None):
        '''
        Args:
            input_ids (torch.Tensor): The input token IDs with shape (B, T)
            attention_mask (torch.Tensor): The attention mask with shape (B, T)

        Returns:
            torch.Tensor: The logits with shape (B, n_classes)

        1. Pass the input token IDs through the embedding layer, encoder, pooler
        2. Apply dropout
        3. Pass the output through the classifier
        '''

        # TODO:

        return logits

We have successfully implemented our own transformer-based classifier for sentiment analysis! Let's train it and see how it compares to the pre-trained one.

In [ ]:
our_model = SimplifiedBertClassifier(vocab_size=VOCAB_SIZE, emb_size=EMB_SIZE, n_layers=N_LAYERS,
                                     n_heads=N_HEADS, intermediate_size=INTERMEDIATE_SIZE, dropout=DROPOUT, n_classes=N_CLASSES)

optimizer = torch.optim.AdamW(our_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

loss_history, acc_history = train(our_model, train_loader, test_loader, optimizer, criterion, device, num_epochs=3, finetuning=False)


In [ ]:
plt.plot(loss_history)
plt.xlabel('Iterations')
plt.ylabel('Cross-entropy Loss')
plt.title('BERT From Scratch, Loss history')
plt.show()

In [ ]:
plt.plot(acc_history)
plt.xlabel('Epochs')
plt.ylabel('Test accuracy')
plt.title('BERT From Scratch, Accuracy history')
plt.show()